### Install Line Bot Sdk

In [ ]:
!pip install line-bot-sdk==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 1.6 MB/s eta 0:00:00


### Install Flask Ngrok for Https Connection

In [ ]:
!pip install Flask==3.0.0 pyngrok==7.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


### Mount to G-Drive for Weights File

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir -p /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

### Intialize Ngrok Connection
This is to create a HTTPS enviornment for LINE bot to communicate back and forth.

In [ ]:
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2024-03-30 11:57:23--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  6.08MB/s    in 2.2s    

2024-03-30 11:57:26 (6.08 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip


In [ ]:
!/ngrok authtoken 2eJUOF6Amanv9Jliv6UXIZPxenW_7uuY6y8wEAyJUeoGMmbP4
!chmod +x /ngrok

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:

ngrok_key = "2eJUOF6Amanv9Jliv6UXIZPxenW_7uuY6y8wEAyJUeoGMmbP4"
port = 5000

In [ ]:
from pyngrok import ngrok


ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

'https://05ae-35-194-232-105.ngrok-free.app'

### Import Libraries

In [ ]:
from flask import Flask, request
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,
    ImageMessage, VideoMessage, AudioMessage
)

In [ ]:
import json
import numpy as np
from io import BytesIO
from PIL import Image
import requests
import os

from google.cloud import storage

import torch
import torchvision.transforms as T
from torchvision.models import resnet50

### LINE Acess Tokens

In [ ]:
CHANNEL_SECRET = 'd6e0a625ea6f530fd43e3e6459597e24'
CHANNEL_ACCESS_TOKEN = 'gjl/0a99GFN1kuY1L1jtBCLrusNphO/Xw9I1DBDNZlVaxlRjrR+uSqwoBJ07YKDASeFRxDEJhG5LBoQ5w8tTFV6K97hEzoV1gM7IVgCFtaGIZqknPEmG07RNREUekR0Xpu9Is5DGmZs2sBqb1Ny/EwdB04t89/1O/w1cDnyilFU='

### Define the Global Variables

In [ ]:
# Define the percentage threshold for replies
UPPER = 60
LOWER = 40

### Load the Model and Transform

In [ ]:
# Set file path
file_path = '/content/drive/MyDrive/The_Turing_Lens/line_bot/resnet50_finetuned_weights_0325_6_800.pth'

model = resnet50(num_classes=4)
model.load_state_dict(torch.load(file_path, map_location=torch.device('cpu')))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# Set the transforms
transform = T.Compose([
    T.ToTensor()
])

### Prediction Function

In [ ]:
def predict(image):

    print('Send in image to transform')
    img_trans = transform(image).unsqueeze(0)
    print('Image transform completed')

    print('Start prediction')

    with torch.no_grad():
        output = model(img_trans)
        prob = torch.softmax(output, dim=1)
        prob = prob.squeeze().numpy()

        prob_dict = {
            'portrait': prob[0],
            'Midjourney': prob[1],
            'Stable Diffusion': prob[2],
            'Bing': prob[3]
        }

    print('Prediction complete')

    portrait_pct = round(prob_dict['portrait'] * 100)

    if portrait_pct >= UPPER:
        ans = f"I'm confident that this is a true portrait with {portrait_pct}% certainty! 📷️"
    elif LOWER <= portrait_pct < UPPER:
        ans = "I recommend validating with caution, as we are not entirely certain if this is a true portrait. 🧐"
    else:
        ans = f"I'm {100 - portrait_pct}% confident that this is an AI-generated portrait. 🤖"

    return ans

### Initialize LINE Bot

In [ ]:
app = Flask(__name__)

@app.route("/callback", methods=['POST'])
def callback():
    body = request.get_data(as_text=True)
    try:
        access_token = CHANNEL_ACCESS_TOKEN
        secret = CHANNEL_SECRET
        json_data = json.loads(body)
        print(json_data)
        line_bot_api = LineBotApi(access_token)
        handler = WebhookHandler(secret)
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)
        if json_data['events'][0]['message']['type'] == 'image':
            image_handler(json_data, line_bot_api)
        else:
            text_handler(json_data, line_bot_api)
    except:
        print(request.args)
    return 'OK'

### Image Handler Function

In [ ]:
# Reply for Image Message here
def image_handler(json_data, line_bot_api):
    msg_id = json_data['events'][0]['message']['id']
    img = line_bot_api.get_message_content(msg_id).content
    reply_token = json_data['events'][0]['replyToken']
    image = Image.open(BytesIO(img))

    try:
        prediction = predict(image)
    except Exception as e:
        prediction = f'Error: {e}'
        print(request.args)

    line_bot_api.reply_message(reply_token, TextSendMessage(prediction))

### Text Handler Function

In [ ]:
# Shows standard reply for message other than images
def text_handler(json_data, line_bot_api):
    reply_message = 'My superpower only works on portrait photos! Send it my way and I\'ll tell you if it\'s a work of art or AI magic! 🪄'
    reply_token = json_data['events'][0]['replyToken']
    line_bot_api.reply_message(reply_token, TextSendMessage(reply_message))

### Initialize Ngrok Service

In [ ]:
if __name__ == "__main__":
  app.run(port = port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'destination': 'U6d55e8b14a33f21feef07dd9c7992b1d', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '501586042374062790', 'quoteToken': 'bcrxFs2ii2E9E5iP6gM-K4sLyUdBhe6IVlyDYCLESyv6O7B1Xi2zjaokYlAUetp79yvBg3npdfkUzyVacu9jdTLAyq670uS0WuuuEKCVQveGg8L_gXLh-M-sgt82NBvkB5WkVQ75wYKkmsOt31_ytA', 'text': 'Hello'}, 'webhookEventId': '01HT7KJN3668HKENC9BGNEXNYN', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711799882832, 'source': {'type': 'user', 'userId': 'U3431c3d148fc32008ca725469c8587da'}, 'replyToken': '811a17cb744742799e904ed7d7532787', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [30/Mar/2024 11:58:03] "POST /callback HTTP/1.1" 200 -


{'destination': 'U6d55e8b14a33f21feef07dd9c7992b1d', 'events': [{'type': 'message', 'message': {'type': 'image', 'id': '501586059017847364', 'quoteToken': 'XghwgCEfqgo38XO0S2Eps5QzOkhNHfO7xn0-2KsTj4B9WG6ch5Mllcnbpb4f6srfQJt1NtpRRz2C3i3hkxDeJXQPgc7h5yF--IGnm8faroJIAUjFb95MURl5NeLiiqyjQ9pCaSfy6SN0UHPNAvAx6g', 'contentProvider': {'type': 'line'}}, 'webhookEventId': '01HT7KJZRQ8027MQS27GG18T53', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711799893569, 'source': {'type': 'user', 'userId': 'U3431c3d148fc32008ca725469c8587da'}, 'replyToken': 'b74dd25ab92742b0a2b860278452f88b', 'mode': 'active'}]}
Send in image to transform
Image transform completed
Start prediction
Prediction complete


...

INFO:werkzeug:127.0.0.1 - - [30/Mar/2024 12:27:35] "POST /callback HTTP/1.1" 200 -


{'destination': 'U6d55e8b14a33f21feef07dd9c7992b1d', 'events': [{'type': 'message', 'message': {'type': 'image', 'id': '501596809085583846', 'quoteToken': 'RhlKGsoEVOQ-Ajy6rkT2DFXBtLJPDFhr4QGGaOCkGwimad1pHxfL-QnHJleW9-Wtt0JaPb8iXc3TtI0TadpFjNZRa82xTSU7QD4Wp-JHVcghxPuylvlOhm-NE1MXko8nMrJvqnl4LuhkAdT533EAOA', 'contentProvider': {'type': 'line'}}, 'webhookEventId': '01HT7SPHEQ7DEPD358V8E5G6S0', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711806301227, 'source': {'type': 'user', 'userId': 'Ue5e628f578f6cdf680eadfb5415f23a7'}, 'replyToken': 'a3a8e4bc8559493ca93becf5f29a7bce', 'mode': 'active'}]}
Send in image to transform
Image transform completed
Start prediction
Prediction complete


INFO:werkzeug:127.0.0.1 - - [30/Mar/2024 13:45:10] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Mar/2024 17:07:44] "POST /callback HTTP/1.1" 200 -


{'destination': 'U6d55e8b14a33f21feef07dd9c7992b1d', 'events': [{'type': 'follow', 'follow': {'isUnblocked': False}, 'webhookEventId': '01HT859PZG8Q5WEQZ2AWWZPSBN', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711818463736, 'source': {'type': 'user', 'userId': 'Uc3bd0702ee64b4f44a562b041d5a09c8'}, 'replyToken': '3ae642e390e84fa58d6a04374be63874', 'mode': 'active'}]}
ImmutableMultiDict([])
{'destination': 'U6d55e8b14a33f21feef07dd9c7992b1d', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '501617224441135330', 'quoteToken': 'yzKSVBchpmF-as_22Dsnhba26gnwWttwFLVWN8Oc1YxSlxxmDcJK3kGpMGm1x8u69l1dzCfDV7URogAXlrkFC2xLGBO9u91dbJ80lBwkRgdhF7fspn2BUU-2HHKylDLHod1RlRraw4Z_pGX451GoYA', 'text': 'Hi '}, 'webhookEventId': '01HT859VRFNZRTVBS24M2K776R', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1711818468814, 'source': {'type': 'user', 'userId': 'Uc3bd0702ee64b4f44a562b041d5a09c8'}, 'replyToken': '5c93ec2382594753b7e72f9caabb22c0', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [30/Mar/2024 17:07:49] "POST /callback HTTP/1.1" 200 -
